# Importing Data

This notebook shows how to conveniently input your data into ComPWA using the python interface!
Also it shows how to calculate kinematic variables.

In [1]:
import pycompwa.ui as pwa
import uproot
import numpy as np

Before we can start, a dummy root data file has to be created. Normally you would skip the next step, since you already have a data file that you want to import.

In [2]:
with uproot.recreate("tuples.root") as f:
    f["data"] = uproot.newtree({
        "D0.px": "float64",
        "D0.py": "float64",
        "D0.pz": "float64",
        "D0.E": "float64",
        "Dm.px": "float64",
        "Dm.py": "float64",
        "Dm.pz": "float64",
        "Dm.E": "float64",
        "pip.px": "float64",
        "pip.py": "float64",
        "pip.pz": "float64",
        "pip.E": "float64",
    })
    f["data"].extend({
        "D0.px": np.array([-0.63207397,  0.11329174, -0.32988257]),
        "D0.py": np.array([-0.02727499,  0.39965045, 0.1848439]),
        "D0.pz": np.array([0.16475981,  0.28701658, -0.37938268]),
        "D0.E": np.array([2.0180088, 1.92316975, 1.93920292]),
        "Dm.px": np.array([0.48782456, 0.08336758, 0.10971082]),
        "Dm.py": np.array([-0.10812153, -0.56649819, -0.13254077]),
        "Dm.pz": np.array([-0.12110724, -0.35891476, 0.44891566]),
        "Dm.E": np.array([1.95351745, 1.96315494, 1.94968155]),
        "pip.px": np.array([0.16118203, -0.10011399, 0.24527202]),
        "pip.py": np.array([0.10152041, 0.19745644, -0.04717316]),
        "pip.pz": np.array([-0.03064159, 0.0802533, -0.04723599]),
        "pip.E": np.array([0.23812766, 0.27373762, 0.28999094]),
    })

First load the data from the root file and bundle the 4 vector data in numpy arrays

In [3]:
tree = uproot.open("tuples.root")["data"]
parts = tree.arrays(['D0*', 'Dm*', 'pip*'])
D0s = np.array((parts[b'D0.px'], parts[b'D0.py'],
                parts[b'D0.pz'], parts[b'D0.E'])).T
Dms = np.array((parts[b'Dm.px'], parts[b'Dm.py'],
                parts[b'Dm.pz'], parts[b'Dm.E'])).T
pis = np.array((parts[b'pip.px'], parts[b'pip.py'],
                parts[b'pip.pz'], parts[b'pip.E'])).T

Now construct ComPWA `Event`s by handing it a list of `FourMomentum`.

In [4]:
events = [pwa.Event(pwa.FourMomentumList([
    pwa.FourMomentum(x[0]),
    pwa.FourMomentum(x[1]),
    pwa.FourMomentum(x[2]),
])) for x in zip(D0s, Dms, pis)]

event_collection = pwa.EventCollection(
    [421, -411, 211],
    pwa.EventList(events)
)
print("Event 0:", event_collection.events[0].four_momenta)

Event 0: FourMomentumList[(-0.632074,-0.027275,2.01801), (0.487825,-0.108122,1.95352), (0.161182,0.10152,0.238128)]


In [5]:
particle_list = pwa.read_particles('../particle_list.xml')
kin = pwa.HelicityKinematics(particle_list, [421], [421, -411, 211])
kin.create_all_subsystems()

2020-03-31 14:39:42,503 [INFO]  Particle a0(980)- with identical ID 9000211 already exists in list with the name a0(980)+ and ID 9000211. Particle properties will be overwritten!
2020-03-31 14:39:42,506 [INFO]  Particle PatricParticle with identical ID 1234 already exists in list with the name Chic1 and ID 1234. Particle properties will be overwritten!
2020-03-31 14:39:42,507 [INFO]  Particle nbar with identical ID 2112 already exists in list with the name n and ID 2112. Particle properties will be overwritten!
2020-03-31 14:39:42,510 [INFO] HelicityKinematics::HelicityKinematics() | Initialized kinematics for reaction ( D0 )->( D0[ID=0] D-[ID=1] pi+[ID=2] )
Event position to final state ID mapping:
0: 0
1: 1
2: 2

2020-03-31 14:39:42,511 [INFO] creating all Subsystems!


Using the `Kinematics.convert()` method, the `EventCollection` can be converted to a `DataSet`, which contains all of the registered kinematic variables (in this example all possible kinematic variables).

In [6]:
dataset = kin.convert(event_collection)
list(dataset.data.keys())

['phi_0_2_vs_1',
 'theta_0_2_vs_1',
 'phi_1_2_vs_0',
 'theta_1_2_vs_0',
 'phi_01_2',
 'theta_0_1_vs_2',
 'mSq_(1,2)',
 'mSq_(0,1,2)',
 'theta_0_12',
 'phi_0_12',
 'mSq_(0,2)',
 'phi_0_1_vs_2',
 'theta_02_1',
 'mSq_(0,1)',
 'theta_01_2',
 'phi_02_1']

In [7]:
dataset.data

{'phi_0_2_vs_1': [1.5380834709116604, -1.9672694162978688, 2.4065009207580226],
 'theta_0_2_vs_1': [0.5840389174806245,
  2.4949116986820825,
  1.2582435108915695],
 'phi_1_2_vs_0': [-1.486876987903377, 1.8321463115593553, -2.7965509151063594],
 'theta_1_2_vs_0': [2.3383862457306104, 0.518132248700785, 1.8577502412492581],
 'phi_01_2': [-2.568328021181151, -1.0719475265076341, 2.94885356067588],
 'theta_0_1_vs_2': [0.6384091290227959,
  2.6102380409321047,
  1.2373291170602045],
 'mSq_(1,2)': [4.35902745213186, 4.789563860203071, 4.69647639184098],
 'mSq_(0,1,2)': [17.719582451928932, 17.29579072179182, 17.46184619639333],
 'theta_0_12': [1.328101671947761, 0.94474023306911, 2.359431809284036],
 'phi_0_12': [-3.1243690623173674, 1.3944020217047697, 2.650939926722521],
 'mSq_(0,2)': [4.844912427657423, 4.334804536424567, 4.761189598045681],
 'phi_0_1_vs_2': [-1.7696963839491957,
  2.1597860942742546,
  -0.24540865118372385],
 'theta_02_1': [1.3163189423118669, 1.0134368628968178, 2.7781